In [ ]:
using Pkg; Pkg.DEFAULT_IO[] = stdout; Pkg.activate("."); Pkg.instantiate();

In [ ]:
Pkg.status()

In [ ]:
using OffsetArrays

L = 2.0
n = 15
dx = L/n

# number of halo points
nh = 2

# a grid with periodic BC
xᶠ = OffsetArray(-nh*dx:dx:L-dx+nh*dx, -nh)
xᶜ = OffsetArray(-nh*dx+dx/2:dx:L-dx/2+nh*dx, -nh);

In [ ]:
u₀(x) = sin(2π/L * x)
h₀(x) = cos(4π/L * x);

In [ ]:
udata = u₀.(xᶠ)
hdata = h₀.(xᶜ);

In [ ]:
∂udata_theoretical =   2π/L * cos.(2π/L * xᶜ)
∂hdata_theoretical = - 4π/L * sin.(4π/L * xᶠ);

In [ ]:
udata

In [ ]:
struct Grid
    L
    n
    dx
    xᶠ
    xᶜ
end

grid = Grid(L, n, dx, xᶠ, xᶜ)

In [ ]:
faces(grid::Grid) = grid.xᶠ[1:grid.n]
cells(grid::Grid) = grid.xᶜ[1:grid.n];

In [ ]:
struct BasicField
    data::AbstractArray
    location::AbstractArray
end

ufield = BasicField(udata, xᶠ)

hfield = BasicField(hdata, xᶜ)

In [ ]:
using Plots

In [ ]:
plot(ufield.location, ufield.data, marker=:circle)
plot!(hfield.location, hfield.data, marker=:square)

Btw, we can add a method for `Plots.plot()` :)

In [ ]:
import Plots: plot, plot!
Plots.plot(f::BasicField, args...; kwargs...) = plot(f.location, f.data, args...; kwargs...)
Plots.plot!(f::BasicField, args...; kwargs...) = plot!(f.location, f.data, args...; kwargs...)

In [ ]:
plot(ufield, marker=:circle)
plot!(hfield, marker=:square)

Let's make a better `Field` type that contains the location of the field as parameter.

In [ ]:
abstract type AbstractLocation end

In [ ]:
struct Cell <: AbstractLocation end 
struct Face <: AbstractLocation end 

In [ ]:
struct Field{L<:AbstractLocation}
    data
    grid
end

In [ ]:
import Plots: plot, plot!
Plots.plot(f::Field{Face}, args...; kwargs...) = plot(f.grid.xᶠ, f.data, args...; kwargs...)
Plots.plot(f::Field{Cell}, args...; kwargs...) = plot(f.grid.xᶜ, f.data, args...; kwargs...)
Plots.plot!(f::Field{Face}, args...; kwargs...) = plot!(faces(f.grid), f.data, args...; kwargs...)
Plots.plot!(f::Field{Cell}, args...; kwargs...) = plot!(cells(f.grid), f.data, args...; kwargs...)

In [ ]:
u = Field{Face}(udata, grid)
h = Field{Cell}(hdata, grid)

Now the types of the fields include information on whether the field lives on cell centers or interfaces. Thus we can write different function methods based on field type.

In [ ]:
typeof(u)

In [ ]:
typeof(h)

Now let's compute derivatives of fields.

In [ ]:
δᶜ(i, u) = u[i+1] - u[i]
δᶠ(i, h) = h[i] - h[i-1];

In [ ]:
δ(i, f::Field{<:Cell}) = δᶠ(i, f.data)
δ(i, f::Field{<:Face}) = δᶜ(i, f.data);

In [ ]:
using BenchmarkTools

In [ ]:
@btime δᶠ(3, h.data)

In [ ]:
@btime δ(3, h)

In [ ]:
∂(i, f::Field{<:Cell}) = δᶠ(i, f.data) / f.grid.dx
∂(i, f::Field{<:Face}) = δᶜ(i, f.data) / f.grid.dx;

In [ ]:
function ∂_arrays(i, ψ::AbstractArray, grid; location="face")
    if location == "face"
        return δᶜ(i, ψ) / grid.dx
    else
        return δᶠ(i, ψ) / grid.dx
    end
    
end

In [ ]:
∂udata = similar(udata)

for i in 1:n
    ∂udata[i] = ∂(i, u)
end

In [ ]:
plot(xᶜ[1:n], ∂udata[1:n])
plot!(xᶜ[1:n], ∂udata_theoretical[1:n])

In [ ]:
@btime ∂(10, h);

In [ ]:
@btime ∂_arrays(10, udata, grid; location="cell");

In [ ]:
∂u = Field{Cell}(similar(u.data), grid)
∂h = Field{Face}(similar(h.data), grid)

In [ ]:
function ∂!(∂f::Field, f::Field)
    @simd for i in 1:f.grid.n
        ∂f.data[i] = ∂(i, f)
    end
end;

In [ ]:
@btime ∂!(∂u, u)

In [ ]:
@btime ∂!(∂h, h)

In [ ]:
plot(∂u, marker=:circle)
plot!(xᶜ[1:n], ∂udata_theoretical[1:n])

In [ ]:
plot(∂h, marker=:circle)
plot!(xᶠ[1:n], ∂hdata_theoretical[1:n])